In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/Vision/faceexp-comparison-data-test-public.csv', error_bad_lines=False)
data.head()

In [ ]:
from PIL import Image
import requests
import cv2
import numpy as np

def get_array_from_im(url):
  r = requests.get(url, allow_redirects=True)
  f = open('image.png', 'wb')
  f.write(r.content)
  image = cv2.imread('image.png')
  if image is not  None:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cascPath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    if len(faces) != 0:
      test_image = Image.open('image.png')
      cropped_example = test_image.crop((faces[0][0], faces[0][1], faces[0][0]+faces[0][2], faces[0][1]+faces[0][3]))
      resize_example = cropped_example.resize((299,299))
      array = np.asarray(resize_example)
      return array
    return None
  return None

In [ ]:
lines_for_data = []
bd = 0
for i, url in enumerate(data['url1']):
  try:
    try:
      r1 = requests.get(url, allow_redirects=True)
      url2 = data['url2'][i]
      r2 = requests.get(url2, allow_redirects=True)
      url3 = data['url3'][i]
      r3 = requests.get(url3, allow_redirects=True)
      if r1.headers.get('content-type') == 'text/html;charset=utf-8' or r2.headers.get('content-type') == 'text/html;charset=utf-8' or r3.headers.get('content-type') == 'text/html;charset=utf-8':
        continue
      labels = [int(data['emotion1'][i]), int(data['emotion2'][i]), int(data['emotion3'][i]), int(data['emotion4'][i]), int(data['emotion5'][i]), int(data['emotion6'][i])]
      labels = np.array(labels)
      label = np.bincount(labels).argmax()
      array1 = get_array_from_im(url)
      array2 = get_array_from_im(url2)
      array3 = get_array_from_im(url3)
      if array1 is None or array2 is None or array3 is None:
        continue
      lines_for_data.append([array1, array2, array3, label])
    except ConnectionResetError:
      bd += 1
  except ValueError:
    bd += 1

In [ ]:
print(len(lines_for_data))

In [ ]:
def preprocess(image):
  first_image = np.reshape(image, (268203,1))
  pixels = []
  for i in first_image:
    pixels.append(i[0])
  return str(pixels).replace('[', '').replace(']', '').replace(',', '')

In [ ]:
new_data = []
for line in lines_for_data:
    new_data.append([preprocess(line[0]), preprocess(line[1]), preprocess(line[2]), line[3]]) 

In [ ]:
df = pd.DataFrame(new_data, columns=['image1', 'image2', 'image3', 'label'])
df.to_csv('/content/drive/My Drive/Vision/test_file.csv')